In [1]:
from mido import MidiFile

import warnings
warnings.filterwarnings("ignore")

In [2]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
import numpy as np
import plotly.express as px
midi_notes = pd.read_csv('midi_notes.csv',sep=";")
midi_scales_chords = pd.read_csv('scales.csv',sep=";")
midi_drum_sounds = pd.read_csv('drums_sounds.csv',sep=";")
midi_drum_sounds.set_index('note', drop=True, inplace=True)
midi_drum_sounds.drop('sound',axis=1, inplace=True)
midi_drum_sounds_dict = midi_drum_sounds.sound_group.to_dict()
midi_instruments = pd.read_csv('instruments.csv',sep=";")

In [3]:
## Levanta el archivo midi, devuelve cantidad de tracks y mensajes contenidos
def load_midi_file(files_mid):
    mid = MidiFile(files_mid, clip=False)
    print('segundos:', mid.length)
    return mid

In [4]:
## Itera sobre los mensaje midi de los tracks con mensjaes y genera un dataframe con notas, velocity, tick_start tick_stop
def get_theme_df(mid):
    dict_notes = {}
    dict_notes_end = {}
    dict_active_notes = {}
    count_notes = 0
    count_notes_end = 0
    last_note_on = 0
    info = []
    controls = []
    key_signatures = []
    time_signatures = []
    instrument_track = []
    n_track = 0
    n_tracks_used = 0
    for track in mid.tracks:
        track_number = 0
        track_name = track.name + str(n_track)
        info.append(track_name)
        n_track = n_track + 1 
        if len(track) > 100:      
            n_tracks_used = n_tracks_used + 1
        time = 0
        has_note_off = any(msg.type == 'note_off' for msg in track)
        for msg in track:          
            if msg.type in ['note_on', 'note_off']:
                time = time + msg.time
                if (has_note_off and (msg.type == 'note_on')) or (not has_note_off and msg.velocity > 0):
                    dict_notes[count_notes] = {"note_num": msg.note, "start": time, "velocity": msg.velocity, "time_dif": time - last_note_on, "track_name": track_number, "channel": msg.channel}
                    dict_active_notes[msg.note] = time
                    count_notes = count_notes + 1
                    last_note_on = time
                else:
                    dict_notes_end[count_notes_end] = {"note_num": msg.note,"track_name": track_number, "start": dict_active_notes[msg.note], "end": time}
                    count_notes_end = count_notes_end + 1
            else:
                if (msg.type == 'control_change'):
                    controls.append(msg.value)
                elif (msg.type == 'key_signature'):
                    key_signatures.append(msg.key)
                elif (msg.type == 'time_signature'):
                    time_signatures.append(str(msg.numerator) + '/' + str(msg.denominator))
                elif (msg.type == 'program_change'):
                    track_number = msg.program
                    instrument_track.append(str(n_track) + "-" + str(msg.program))
    tema_df = pd.DataFrame.from_dict(dict_notes, "index")
    tema_df_notes_end = pd.DataFrame.from_dict(dict_notes_end, "index")
    tema_df_merged = pd.merge(tema_df, tema_df_notes_end,on=['note_num','start','track_name'])
    controls = pd.Series(controls).head(40)
    controls = controls[controls > 10].sum() / n_tracks_used
    return tema_df_merged, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track                 

In [5]:
## Reemplaza outlyers de duración
def limit_outlyer_duration_notes(tema_df):
    notes_weight = pd.cut(tema_df.duration, 6)

    outlyeras_duration = pd.DataFrame(tema_df.duration.quantile([0.05,0.95]))

    mask_outlyers_lower = tema_df.duration < outlyeras_duration.duration[0.05]
    tema_df.loc[mask_outlyers_lower,'duration'] = outlyeras_duration.duration[0.05]

    mask_outlyers_higher = tema_df.duration > outlyeras_duration.duration[0.95]
    tema_df.loc[mask_outlyers_higher,'duration'] = outlyeras_duration.duration[0.95]

    notes_weight = pd.cut(tema_df.duration, 6)
    return tema_df

In [6]:
# Crea una mascara para limitar el ploteo en el tiempo
def plot_theme(data_to_plot, plot_from_second=0, plot_to_second=30, size_plot='duration', color_plot='grupos_track'):
    ## Poltea las notas en el tiempo
    mask_tiempo_from = (tema_df.segundo >= plot_from_second)
    mask_tiempo_to =  (tema_df.segundo <= plot_to_second)
    fig = px.scatter(data_frame = tema_df[mask_tiempo_from & mask_tiempo_to], x = "segundo", y = "note_num",
                     size = size_plot, color = color_plot, opacity = 1, color_discrete_sequence=None, labels='note', height=600,)

    fig.show()

In [7]:
## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
def cuantize(tema_df):
    tema_df2_mask = (tema_df.iloc[1:len(tema_df),:].note_simultaneous == True)
    tema_df2_mask = tema_df2_mask
    tema_df2_mask[0] = False

    tema_df.loc[tema_df2_mask, 'start'] = np.NaN
    tema_df.start = tema_df.start.fillna(method='ffill')
    tema_df.start = tema_df.start.astype(int)
    return tema_df

In [8]:
def restore_df():
    tema_df = tema_df_copy

In [9]:
def generate_groups(tema_df):
    ## Cada nota l y x l van a tener mas puntos que despues van a ser removidos
    tema_df.loc[:,'note_fake'] = 0
    long_notes_mask = tema_df.cat_duration.apply(lambda x: x in ['l','xl'])
    fake_notes = tema_df[long_notes_mask]
    fake_notes.start = fake_notes.apply(lambda x: x.start + (x.end - x.start) / 2 if x.cat_duration == 'l' else x.start + (x.end - x.start) / 3,axis=1)

    fake_notes.duration = fake_notes.duration / 2
    fake_notes.note_fake = 1

    tema_df = pd.concat([tema_df,fake_notes, fake_notes_lx]).sort_values('start')
    
    xlong_notes_mask = tema_df.cat_duration == 'xl'
    fake_notes_lx = tema_df[xlong_notes_mask]
    fake_notes_lx.start = fake_notes_lx.start + (fake_notes_lx.end - fake_notes_lx.start)  / 3 * 2
    fake_notes_lx.duration = fake_notes_lx.duration / 2
    fake_notes_lx.note_fake = 1

    ## Implementacion DBSCAN para encontrar grupos de notas y segmentarlas
    from sklearn.cluster import DBSCAN
    from sklearn.preprocessing import StandardScaler

    X = tema_df[['start', 'note_num']]
    X.loc[:,'start'] = X.start / 15
    X.loc[:,'note_num'] = X.note_num / 0.2

    X = X.values

    # Implementación de DBSCAN
    dbscan = DBSCAN( eps=29, min_samples=3)
    y_dbscan = dbscan.fit_predict(X)
    y_dbscan

    labels = dbscan.labels_
    tema_df.loc[:,'grupos'] = labels
    cant_grupos = pd.Series(labels).unique().shape[0]

    ## Concatena los grupos creados con DBSCAN con las cadenas de notas recibidas en cada track
    tema_df.loc[:,'grupos_track'] = tema_df.grupos.astype(str) + "_" + str(tema_df.track_name)
    #plot_theme(tema_df)

    ## elimina notas falsas
    notes_fake_mask = tema_df.note_fake == 1
    tema_df.drop(tema_df[notes_fake_mask].index, inplace=True)

    ## Imprime el dataset completo ordenado por grupos descartando el ruido
    ## Calcula el ruido
    mask_noise = tema_df.grupos != -1
    tema_df.sort_values(['grupos_track','start'],ascending=[True,True],inplace=True)
    return tema_df, cant_grupos

In [24]:
def get_theme_stats(file_path, composer, file_name):
    ## Instancia el archivo midi
    mid = load_midi_file(file_path)
    duracion_tema = mid.length
    tema_df, info, controls, key_signatures, time_signatures, n_tracks_used, instrument_track = get_theme_df(mid)  

    ## Parsea a enteros los valores numéricos
    for col in tema_df.loc[:,tema_df.columns!="track_name"].columns:
        tema_df[col] = pd.to_numeric(tema_df[col])
    
    ## Calcula la duración de cada nota
    tema_df.loc[:,'duration'] = tema_df.end - tema_df.start

    
    ## Agregamos informacion de instrumentos y batería
    tema_df = pd.merge(tema_df, midi_instruments,how='left',left_on='track_name',right_on='num_mid').drop('num_mid',axis=1)
    tema_df.loc[tema_df.channel == 9,['intrument_subcat']] = tema_df[tema_df.channel == 9].note_num.apply(lambda x: midi_drum_sounds_dict[x])
    tema_df.loc[tema_df.channel == 9,['intrument_cat']] = 'Drums'
    
    ## Generamos datos estadisticos de la instrumentación
    instrumentos_por_seg = pd.Series(tema_df.intrument_subcat.value_counts() / duracion_tema)
    
    ## Eliminamos las notas de batería de nuestro analisis musical 
    tema_df = tema_df.loc[tema_df.intrument_cat != 'Drums']

    ## agrega el nombre y octava de notas a la tabla
    tema_df = pd.merge(tema_df, midi_notes,how='left',left_on='note_num',right_on='note_number').drop('note_number',axis=1)

    ##elimina notas demasiado cortas y demasiado largas que pueden afectar al análisis
    tema_df = limit_outlyer_duration_notes(tema_df)

    ## Categoriza duración

    cat_duration = pd.cut(tema_df.duration, 6, labels=['xxs','xs','s','m','l','xl'])
    tema_df.loc[:,'cat_duration'] = cat_duration

    ## Categoriza VELOCITY
    cat_velocity = pd.cut(tema_df.velocity, 6, labels=['pp','p','m','mf','f','ff'])
    tema_df.loc[:,'cat_velocity'] = cat_velocity

    ## Describe, muchos de estos valores van a ser utiles como predictores
    tema_describe = tema_df.describe()

    ## Cuantificación de notas:
    ## define cuales notas están tan cera que deberían estar en el mismo instante

    unit_duplicate = tema_describe.loc['min','duration'] / 1.5
    tema_df.loc[:,'note_simultaneous'] = tema_df.time_dif < unit_duplicate

    ## Reemplaza los tiempos de notas muy cercanas por notas simultaneas
    #tema_df = cuantize(tema_df)

    ## calcula la cantidad de ticks x segundo
    ticks_por_seg = tema_df.end.max() / duracion_tema

    ## Calcula la cantidad de notas que existen en simultaneo
    tema_df_simultaneous =  tema_df.start.value_counts()
    tema_df_simultaneous_times = tema_df_simultaneous.loc[tema_df_simultaneous > 1].index.to_list()

    tema_df.note_simultaneous = False
    tema_df.loc[:,'note_simultaneous'] = tema_df.start.apply(lambda x: True if x in tema_df_simultaneous_times else False) 

    ## Convierte unidad de medida de timpo Ticks a segundos en cada nota
    tema_df.loc[:,'segundo'] = tema_df.start / ticks_por_seg

    #generate_groups(tema_df)
    
    ## Shape final del dataset
    notas_totales = tema_df.shape[0]

    ## indice de actividad (cantidad de notas) por tiempo
    cant_eventos_individuales = (notas_totales - len(tema_df_simultaneous_times) / 2)
    actividad_por_tiempo = cant_eventos_individuales / duracion_tema
    velocity_avg = tema_df.cat_velocity.value_counts(normalize=True)
    length_notes_avg = tema_df.cat_duration.value_counts(normalize=True)

    ## Analiza proporciones de notas y duraciones mas repetidas
    notes_weight = round(tema_df.note.value_counts(normalize=True) * 100,2)

    notes_weight = round(tema_df.note_name.value_counts(normalize=True) * 100,2)
    all_values_notes = pd.DataFrame(notes_weight).reset_index()
    most_probable_scale = all_values_notes.head(7)
    scale_coverage = notes_weight.head(7).sum()
    avr_vertical_notes = tema_df.note_simultaneous.sum() / notas_totales
    cant_pedal_sustain = controls
    cant_eventos_por_pedal = cant_eventos_individuales / cant_pedal_sustain if cant_pedal_sustain > 5 else np.NaN
    cant_pedales_seg = cant_pedal_sustain / duracion_tema if cant_pedal_sustain > 5 else np.NaN

    #obtiene informacion de la escala
    nombre_escala = pd.merge(most_probable_scale, midi_scales_chords, how='left', left_on='index', right_on='note_name')
    nombre_escala.fillna(0,inplace=True)
    nombre_escala_T = nombre_escala.T
    nombre_escala_T
    mask = nombre_escala_T.apply(lambda x: True if all(x != 0) else False, axis=1)
    mask
    tabla_esacla = nombre_escala_T[mask].T

    tabla_esacla

    nombre_columna_Tmaj = tabla_esacla.columns[3]
    tonalidad  = 0
    tonalidad_escala = 'M'
    dic = {6:1, 7:2, 1:3, 2:4, 3:5, 4:6, 5:7}
    if nombre_columna_Tmaj != "U":
        tabla_esacla.set_index(tabla_esacla.columns[3],inplace=True,drop=False)
        mayor_chord_coverage = tabla_esacla.loc[[1,3,5],:'note_name_x'].sum()[1]
        minor_chord_coverage = tabla_esacla.loc[[6,1,3],:'note_name_x'].sum()[1]


        if minor_chord_coverage > mayor_chord_coverage:
            tonalidad = tabla_esacla.iloc[0,2]
            # reemplazo el 6to grado por la tónica
             # define desired replacements here

            tabla_esacla[nombre_columna_Tmaj] = tabla_esacla[nombre_columna_Tmaj].apply(lambda x: dic[x])
            tabla_esacla
            tonalidad_escala = 'm'
            tabla_esacla.set_index(nombre_columna_Tmaj,inplace=True,drop=False)
            tabla_esacla.sort_index(inplace=True)
        else:
            tonalidad = nombre_columna_Tmaj

    elif len(key_signatures) > 0:
        if 'b' in key_signatures[0]:
            dict_keys = {'Db':'C#', 'Eb':'D#', 'Gb':'F#', 'Ab':'G#', 'Bb':'A#','Dbm':'C#m', 'Ebm':'D#m', 'Gbm':'F#m', 'Abm':'G#m', 'Bbm':'A#m'}
            tonalidad =  dict_keys[key_signatures[0]]
        else:
            tonalidad = key_signatures[0]

        midi_scales_chords_weighted = pd.merge(midi_scales_chords[['note_name', tonalidad]], all_values_notes, how='left', left_on='note_name', right_on='index',)
        midi_scales_chords_weighted.drop('index',axis=1,inplace=True)
        midi_scales_chords_weighted.columns = ['note_name', 'scale', 'weight']
        midi_scales_chords_weighted.set_index(midi_scales_chords_weighted.columns[1],inplace=True,drop=False)
        if 'm' in key_signatures[0]:
            mayor_chord_coverage = midi_scales_chords_weighted.loc[[3,5,7],'weight'].sum()
            minor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
        else:
            mayor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
            minor_chord_coverage = midi_scales_chords_weighted.loc[[6,1,3],'weight'].sum()        
            if minor_chord_coverage > mayor_chord_coverage:
                tonalidad_escala = 'm'
                midi_scales_chords_weighted_mask = midi_scales_chords_weighted.scale > 0
                midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'] = midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'].apply(lambda x: dic[x])
        midi_scales_chords_weighted.sort_index(inplace=True)
        midi_scales_chords_weighted.drop('scale',axis=1,inplace=True)
        scale_coverage = midi_scales_chords_weighted.head(7).sum()[1]
        tabla_esacla = midi_scales_chords_weighted
        tabla_esacla
    else:
        tonalidad = 'U'
        tonalidad_escala = 'U'
        tabla_esacla = pd.DataFrame(columns=['1','2'])
        tabla_esacla.iloc[0,7,:] = 0


    ## crea el Music stats    
    music_stats = pd.DataFrame(columns=['time_signatures','tonalidad','tonalidad_escala','scale_coverage','mayor_chord_coverage','minor_chord_coverage',
                                  'sc_1','sc_2','sc_3','sc_4','sc_5','sc_6','sc_7', 'avr_vertical_notes','cant_eventos_por_pedal',
                                  'cant_pedales_seg','duracion_seg','tracks_used', 'info_tracks'])
    music_stats.loc[0] = [time_signatures, tonalidad, tonalidad_escala, scale_coverage,  mayor_chord_coverage, minor_chord_coverage, 
                   tabla_esacla.iloc[0,1], tabla_esacla.iloc[1,1], tabla_esacla.iloc[2,1], 
                   tabla_esacla.iloc[3,1], tabla_esacla.iloc[4,1], tabla_esacla.iloc[5,1], 
                   tabla_esacla.iloc[6,1], avr_vertical_notes,cant_eventos_por_pedal,
                   cant_pedales_seg,duracion_tema,n_tracks_used,
                  " ".join(info)]
      
    tema_describe = tema_df.describe()
    
    data_describe = pd.DataFrame(tema_describe.loc[tema_describe.index != 'count',['note_num','Octave','duration']].unstack())
    data_describe.reset_index(inplace=True)
    data_describe.loc[:,'name'] = data_describe.level_0 + "_" + data_describe.level_1
    data_describe.set_index('name',inplace=True)
    data_describe.drop(['level_0','level_1'],axis=1, inplace=True)
    
    ## Agrego informacion de genero y grupo tomado de los archivos
    array_dir = file_path.split('\\')
    labels_mid = pd.DataFrame(data=[[array_dir[6],array_dir[7],array_dir[len(array_dir) - 1].replace('.mid','')]], columns=['Genero','Grupo','tema']).T
    print(labels_mid)
    df_final = pd.concat([music_stats.T,instrumentos_por_seg, data_describe,velocity_avg,length_notes_avg, labels_mid])
    return df_final.to_dict()

In [25]:
import os
classical = 'C:\DH\ds_blend_students_2020\desafio IV\MIDI folders\midi_selection'

# the dictionary to pass to pandas dataframe
dict = {}
count_files = 0
      
for root, dirs, files in os.walk(classical, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file, "compositor": 'All'}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
display(df_files)

,file,file_name,compositor
0,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,01 Menuet.mid,ClassicRock
1,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,02 Menuet.mid,ClassicRock
2,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,03 Menuet.mid,ClassicRock
3,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,04 Menuet.mid,ClassicRock
4,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,05 Polonaise.mid,ClassicRock
...,...,...,...
5805,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,REGGAE-2.MID,ClassicRock
5806,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,REGGA~16.MID,ClassicRock
5807,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,CLOSE-~6.MID,ClassicRock
5808,C:\DH\ds_blend_students_2020\desafio IV\MIDI f...,SWEET-~6.MID,ClassicRock


In [30]:
df_files_analize = df_files
df_files_analize.shape

(5810, 3)

In [31]:
dict = {}
count_files = 0

In [ ]:
from contextlib import suppress

errors_arr = []
for index, row in df_files_analize.iterrows():
    with suppress(Exception):
        dict[count_files] = get_theme_stats(row.file, row.compositor, row.file_name)[0]
        count_files = count_files + 1
        print('success', str(count_files))

df_midi_stats = pd.DataFrame.from_dict(dict, "index")    


from contextlib import suppress

errors_arr = []
for index, row in df_files_analize.iterrows():
    dict[count_files] = get_theme_stats(row.file, row.compositor, row.file_name)[0]
    count_files = count_files + 1
    print('success', str(count_files))

df_midi_stats = pd.DataFrame.from_dict(dict, "index")    

In [33]:
df_midi_stats.to_csv('midi_stats_new.csv')

In [34]:
df_midi_stats

,time_signatures,tonalidad,tonalidad_escala,scale_coverage,mayor_chord_coverage,minor_chord_coverage,sc_1,sc_2,sc_3,sc_4,...,Fiddle,Tinkle Bell,Bagpipe,Cuica,Dulcimer/Santur,Woodblock,Square,Koto,Blown Bottle,Shamisen
0,[3/4],F,M,93.52,44.66,41.99,17.18,15.65,14.89,14.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[3/4],G,M,97.66,56.54,47.66,22.43,18.69,15.42,14.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[3/4],D,m,93.79,43.54,50.72,16.27,13.88,14.83,12.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[3/4],G,M,98.40,62.62,52.39,26.84,20.13,15.65,9.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[3/4],F,M,98.46,54.14,43.16,19.46,18.50,16.18,15.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5134,[4/4],G,M,98.81,62.58,47.17,26.48,25.16,12.38,10.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5135,[4/4],A,M,99.66,60.65,44.49,22.59,22.07,16.85,15.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5136,[4/4],A#,m,99.52,55.17,59.77,22.41,4.90,10.10,13.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5137,[4/4],C,M,98.75,64.89,32.36,41.50,14.13,12.11,11.67,...,NaN,NaN,NaN,0.006991,1.083613,NaN,NaN,NaN,NaN,NaN
